In [1]:
import geopandas as gpd
import pandas as pd
import fiona
import os
import matplotlib.pyplot as plt
import folium
from zipfile import ZipFile
from folium.plugins import MarkerCluster, HeatMap, BeautifyIcon
from folium.map import LayerControl, Layer, FeatureGroup
import seaborn as sns
from shapely.geometry import Point, LineString, MultiPoint
import numpy as np
import contextily as ctx
import requests
from io import StringIO, BytesIO
import json
import datetime as dt
from ast import literal_eval
from shapely.wkt import loads
import plotly.express as px
from dotenv import load_dotenv, find_dotenv

In [2]:
# API KEY
load_dotenv(find_dotenv())
GOOGLE = os.environ.get("GOOGLE")

In [40]:
# first find all restaurants

# circle:radius@lat,lng.
url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'
params = {            
            'location':'45.05917,7.67899', #sensor
            'radius':'2000',
            'type':'bar',
            'key':GOOGLE,
#             'next_page_token':'Aap_uEAHLKFqkT_9cbZmb1MQnETt-m8UbzadW7QiNATmacn1_n9Po988eGWZ5DmGW8qTzeghWKZ_IHX32S5nIapSSvITOgSual649oOVvz-ODoWvcFpMs78Wx8XZhoGqtpuvgVqcmk_Ra0QA7DEUSuOdhuVoLcjfX1fKFDUxqNM_6kqN48VxN4PWKM9ScxwjzrSnMjPPtkmAxCpmIp_yn9BTZq4kY33tCoUS0DPJs52ZLSpMAmIZN4NdPdfq28l8biQCmJtYEStMHtOJB9HeL0nEyyD4NJeKh_SrK-oyT2IUHvqaCi8VWlOr7TG-ubGlj-k9mu5MGm6es_gQLijxhkargh4Fsx_1PBmNqARGefGflOUyjIOiTulxf2qMVPtmDltGALkT482L21uUi1UpvfUxr9ytQnRSNGfSm0W1j6mrX28O7iNa3Ac'
         } 

r = requests.get(url, params=params)
# resp = r.json()
# https://maps.googleapis.com/maps/api/place/findplacefromtext/json?input=mongolian%20grill&inputtype=textquery&fields=photos,formatted_address,name,opening_hours,rating&locationbias=circle:2000@47.6918452,-122.2226413&key=YOUR_API_KEY



In [41]:
results = r.json().get('results')


In [38]:
results2 = r.json().get('results')

In [44]:
bars = results + results2

In [132]:
bars[2]

{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 45.066405, 'lng': 7.684872},
  'viewport': {'northeast': {'lat': 45.0676660802915,
    'lng': 7.686084630291502},
   'southwest': {'lat': 45.0649681197085, 'lng': 7.683386669708497}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/restaurant-71.png',
 'name': 'Japs!',
 'opening_hours': {'open_now': False},
 'photos': [{'height': 3968,
   'html_attributions': ['<a href="https://maps.google.com/maps/contrib/115455084168180704575">Noemi Vagelatos</a>'],
   'photo_reference': 'Aap_uEDz6ShOIZADYb0nWarfM7RGt6bwmJ94nqO0uo67NCOxlgrxhM5fEPvaKuBvU63_oZdbjxv4ArAu2Mjo5hF9q7c08LX7zfYbqI4pFTXYHQ5L7HBNNwKrZrR4uoR4UQ9xye2ewNL6grY9Ry2PdaXHyyRnU_rUzxiK2JDD9C3d86gmees',
   'width': 2976}],
 'place_id': 'ChIJYZuin29tiEcRwqEvNL2U5Hc',
 'plus_code': {'compound_code': '3M8M+HW Turin, Metropolitan City of Turin, Italy',
  'global_code': '8FQ93M8M+HW'},
 'price_level': 2,
 'rating': 4.4,
 'reference': 'ChIJYZuin29tiEcR

In [63]:
url = 'https://maps.googleapis.com/maps/api/place/details/json'
params = {
    'key':GOOGLE,
    'fields':'opening_hours'
         }
opening_hrs = []
for bar in bars:
    reference = bar.get('reference')
    params['place_id'] = reference
    r = requests.get(url, params=params)
    opening_hrs.append(r)


In [66]:
contents_hrs = [r.json() for r in opening_hrs]
periods = []
for x in contents_hrs:
    try:
        hr = x.get('result').get('opening_hours').get('periods')
        periods.append(hr)
    except:
        pass


In [177]:
# remove 24h open bars
new = [x for x in periods if len(x) > 1]

In [139]:
contents_hrs[11]

{'html_attributions': [],
 'result': {'opening_hours': {'open_now': False,
   'periods': [{'close': {'day': 2, 'time': '0430'},
     'open': {'day': 1, 'time': '2200'}},
    {'close': {'day': 3, 'time': '0430'}, 'open': {'day': 2, 'time': '2200'}},
    {'close': {'day': 4, 'time': '0430'}, 'open': {'day': 3, 'time': '2200'}},
    {'close': {'day': 5, 'time': '0430'}, 'open': {'day': 4, 'time': '2200'}},
    {'close': {'day': 6, 'time': '0430'}, 'open': {'day': 5, 'time': '2200'}},
    {'close': {'day': 0, 'time': '0430'}, 'open': {'day': 6, 'time': '2200'}}],
   'weekday_text': ['Monday: 10:00 PM – 4:30 AM',
    'Tuesday: 10:00 PM – 4:30 AM',
    'Wednesday: 10:00 PM – 4:30 AM',
    'Thursday: 10:00 PM – 4:30 AM',
    'Friday: 10:00 PM – 4:30 AM',
    'Saturday: 10:00 PM – 4:30 AM',
    'Sunday: Closed']}},
 'status': 'OK'}

In [224]:
closing = []
for x in new:
    for i in x:
        _close = i.get('close')
        closing.append(_close)
opening = []
for x in new:
    for i in x:
        _open = i.get('open')
        opening.append(_open)


In [225]:
# put results in dataframe
opening_times = pd.DataFrame(opening)
closing_times = pd.DataFrame(closing)
closing_times['time'] = pd.to_datetime(closing_times['time'], format='%H%M')
opening_times['time'] = pd.to_datetime(opening_times['time'], format='%H%M')

In [226]:
# create unique day_hr identifier
closing_times['day_time'] = closing_times.apply(lambda x: str(x.day) + '_' + str(x.time.hour), axis=1)
opening_times['day_time'] = opening_times.apply(lambda x: str(x.day) + '_' + str(x.time.hour), axis=1)

In [232]:
# count all apperances of openings and closings per weekday
agg_close = closing_times.groupby('day_time').agg({'day':'count'}).rename(columns={'day':'count_close'})
agg_open = opening_times.groupby('day_time').agg({'day':'count'}).rename(columns={'day':'count_open'})
agg_joint = agg_close.join(agg_open, how='outer')

In [236]:
# init range 2018
range_2018 = pd.DataFrame(pd.date_range('2018-01-01', '2018-12-31', freq='h'), columns=['hour'])
range_2018['day_time'] =  range_2018.apply(lambda x: str(x.hour.weekday()) + '_' + str(x.hour.hour), axis=1)

In [240]:
# join both

opening_count_2018 = range_2018.merge(agg_joint, 
                                    on='day_time',
                                    how='left').drop(columns='day_time')

In [242]:
opening_count_2018.to_csv('raw_data/opening_count_2018.csv')